In [1]:
import json
import pandas as pd
import numpy as np
import time
import requests

logic:
1. at the last second block in current epoch, get the effective stakes in current epoch (effective stakes donot change during the whole epoch, and we cannot get the effective stake info at the last block, so we choose the last second)
2. at the last block in current epoch, get the block reward (accumulated reward in current block - acculated reward in last block)
3. at the first block in new epoch, get the new effective stakes in new epoch, check the effective stakes changes. get the block reward, check the difference of block reward.
4. check if the effective stake increases, whether the block reward increases, if the effective stake decreases, whether the block reward decreases.

In [2]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [3]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [4]:
def getLastBlockOfCurrentEpoch():
    method = 'hmy_getStakingNetworkInfo'
    params = []
    return get_information(method, params)['result']['epoch-last-block']

In [5]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [0]
    return get_information(method, params)['result'] 

In [6]:
def getStakeAndRewards():
    stake = dict()
    reward = dict()
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['currently-in-committee'] == True:
            address = i['validator']['address']
            reward_accumulated = i['lifetime']['reward-accumulated']
            effective_stake = float(i['metrics']['total-effective-stake'])
            stake[address] = effective_stake
            reward[address] = reward_accumulated
    return stake, reward

In [7]:
def getRewards():
    reward = dict()
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['currently-in-committee'] == True:
            address = i['validator']['address']
            reward_accumulated = i['lifetime']['reward-accumulated']
            reward[address] = reward_accumulated
    return reward

In [8]:
def diffAndFilter(map1, map2):
    map3 = dict()
    for k, v in map2.items():
        if k in map1:
            if v - map1[k] != 0:
                map3[k] = v - map1[k]
    return map3

In [9]:
def R11_test():
    print("Test-R11: Earning is proportional to effective stake ")
    num = int(input("enter the number of epoches you want to test: "))
    
    iterations = 0
    while iterations < num:
        print("test " + str(iterations+1) + " will begin ...")
        block = getBlockNumber()
        last_block = getLastBlockOfCurrentEpoch()
        if block == last_block:
            new_block = block+1
            while block < new_block:
                block = getBlockNumber()
        print("current and last block numbers", block, last_block)
        second_last_block = last_block - 1
        while block < second_last_block:
            block = getBlockNumber()
        print("second last block in current epoch reached", block, "will begin testing...")
        stakes, rewards = getStakeAndRewards()
        
        # in the last block, we can not get the total effective stakes, no metrics. 
        new_block = block + 1
        while block < new_block:
            block = getBlockNumber()
        print("last block in current epoch reached", block, "will calculate the reward per block")
        next_rewards = getRewards()
        reward_per_block = diffAndFilter(rewards, next_rewards)

        new_block = block + 1
        while block < new_block:
            block = getBlockNumber()
        print("first block in new epoch reached", block, "will calculate the difference of reward per block and difference of effective stake")
        new_stakes, new_rewards = getStakeAndRewards()
        
        reward_per_block_next = diffAndFilter(next_rewards, new_rewards)
        reward_diff = diffAndFilter(reward_per_block, reward_per_block_next)
        print("reward diff: ", reward_diff)
        # get the validators whose effective stake changes
        stake_diff = diffAndFilter(stakes, new_stakes)

        if not stake_diff:
            print("in this iteration, no validators change the effective stake")
        if not reward_diff:
            print("in this iteration, no validators change the reward")
            
        for k,v in stake_diff.items():
            if k in reward_diff:
                if v > 0: 
                    if reward_diff[k] <= 0:
                        print("Test-R11: Fail")
                        print("effective stake increase: ", stake_diff)
                        print("reward per block doesn't increase: ", reward_diff)
                    else:
                        print("Test-R11: Success")
                if v < 0:
                    if reward_diff[k] >= 0:
                        print("Test-R11: Fail")
                        print("effective stake decrease: ", stake_diff)
                        print("reward per block doesn't decrease: ", reward_diff)
                    else:
                        print("Test-R11: Success")
                
        iterations += 1        

In [10]:
R11_test()

Test-R11: Earning is proportional to effective stake 
enter the number of epoches you want to test: 1
test 1 will begin ...
current and last block numbers 1971 1975
second last block in current epoch reached 1974 will begin testing...
last block in current epoch reached 1975 will calculate the reward per block
first block in new epoch reached 1976 will calculate the difference of reward per block and difference of effective stake
reward diff:  {}
in this iteration, no validators change the reward


In [11]:
def getValidatorInfo(validator):
    method = "hmy_getValidatorInformation"
    params = [validator]
    return get_information(method, params)['result']

In [12]:
i = 'one1tpxl87y4g8ecsm6ceqay49qxyl5vs94jjyfvd9'
block = getBlockNumber()
last_block = getLastBlockOfCurrentEpoch()
new_block = last_block + 1
while block <= new_block:
    if block == last_block or block == last_block-1:
        print("meet the last block, not check")
        next_block = block + 1
        while block < next_block:
            block = getBlockNumber()
        continue
    a = float(getValidatorInfo(i)['metrics']['total-effective-stake'])
    print("current block number: ", block, "current effective stake: ", a)
    
    next_block = block + 1
    while block < next_block:
        block = getBlockNumber()
    b = float(getValidatorInfo(i)['metrics']['total-effective-stake'])
    print("new block number: ", block, "new effective stake: ", b)
    stake = b-a
    print("difference of effective stake: ", stake)
    

current block number:  1976 current effective stake:  1.38e+24
new block number:  1977 new effective stake:  1.38e+24
difference of effective stake:  0.0
current block number:  1977 current effective stake:  1.38e+24
new block number:  1978 new effective stake:  1.38e+24
difference of effective stake:  0.0
current block number:  1978 current effective stake:  1.38e+24
new block number:  1979 new effective stake:  1.38e+24
difference of effective stake:  0.0
current block number:  1979 current effective stake:  1.38e+24
new block number:  1980 new effective stake:  1.38e+24
difference of effective stake:  0.0
current block number:  1980 current effective stake:  1.38e+24
new block number:  1981 new effective stake:  1.38e+24
difference of effective stake:  0.0
current block number:  1981 current effective stake:  1.38e+24
new block number:  1982 new effective stake:  1.38e+24
difference of effective stake:  0.0
current block number:  1982 current effective stake:  1.38e+24
new block num

In [13]:
i = 'one1tpxl87y4g8ecsm6ceqay49qxyl5vs94jjyfvd9'
block = getBlockNumber()
last_block = getLastBlockOfCurrentEpoch()
new_block = last_block + 1
while block <= new_block+1:
    block = getBlockNumber()
    a = getValidatorInfo(i)['lifetime']['reward-accumulated']
    next_block = block +1
    while block < next_block:
        block = getBlockNumber()
    b = getValidatorInfo(i)['lifetime']['reward-accumulated']
    reward = b-a
    print("current block", block, "current earning", reward)


current block 2016 current earning 8024922118380062336
current block 2017 current earning 8024922118380062336
current block 2018 current earning 8024922118380062336
current block 2019 current earning 8024922118380062336
current block 2020 current earning 8024922118380062336
current block 2021 current earning 8024922118380062336
current block 2022 current earning 8024922118380062336
current block 2023 current earning 8024922118380062336
current block 2024 current earning 8024922118380062336
current block 2025 current earning 8024922118380062336
current block 2026 current earning 8024922118380062336
current block 2027 current earning 8024922118380062336
current block 2028 current earning 8024922118380062336
current block 2029 current earning 8024922118380062336
current block 2030 current earning 8024922118380062336
current block 2031 current earning 8024922118380062336
current block 2032 current earning 8024922118380062336
current block 2033 current earning 8024922118380062336
current bl